# K2 Platform - Binance Live Streaming E2E Demo

**Phase 2 Prep Achievement**: Complete Binance Cryptocurrency Streaming Pipeline

This notebook demonstrates the complete end-to-end data pipeline for live cryptocurrency market data streaming, showcasing the achievements of Phase 2 Prep:

```
┌─────────────────────────────────────────────────────────────────────┐
│                  K2 Binance Streaming Architecture                  │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│  Binance WebSocket → Kafka (Avro) → Consumer → Iceberg → Query     │
│      (Live)            (Stream)      (Batch)    (ACID)    (DuckDB) │
│                                                                     │
│  • Real-time crypto trades (BTC, ETH, BNB)                          │
│  • V2 hybrid schema with vendor_data map                            │
│  • Multi-source capability (ASX batch + Binance streaming)          │
│  • Multi-asset-class platform (equities + crypto)                   │
│  • Production-grade resilience (SSL, metrics, circuit breakers)     │
│  • 138 msg/s throughput, sub-second queries                         │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

## What This Demo Covers

1. **Infrastructure Setup** - Docker services validation and initialization
2. **Live Streaming** - Connect to Binance WebSocket, stream real trades
3. **Kafka Pipeline** - Message broker with partitioning and Avro serialization
4. **Consumer Processing** - Batch processing from Kafka to Iceberg (ACID)
5. **Iceberg Lakehouse** - Query 5,000+ trades with DuckDB
6. **Real-Time Visualizations** - Price charts, volume analysis, metrics
7. **Data Quality** - Schema validation, sequence tracking, duplicate detection
8. **Vendor Data** - Exchange-specific fields preserved in JSON

## Learning Objectives

By the end of this notebook, you will understand:
- How to connect to live cryptocurrency exchanges via WebSocket
- How to build a production-ready streaming data pipeline
- How v2 hybrid schemas enable multi-source compatibility
- How to achieve ACID guarantees with Apache Iceberg
- How to query streaming data with sub-second latency
- How to validate data quality in real-time systems

**Estimated Time**: 30-45 minutes

## 2. Prerequisites & Setup Overview

### System Requirements

- ✓ **Docker** (with Docker Compose) - Running all infrastructure services
- ✓ **Python 3.12+** - Managed via `uv` package manager
- ✓ **4GB RAM minimum** - For Kafka, Iceberg, MinIO, PostgreSQL
- ✓ **Internet connection** - For Binance WebSocket streaming
- ✓ **Disk space** - ~2GB for Docker images and data

### Docker Services Required

| Service | Purpose | Port |
|---------|---------|------|
| Kafka | Message broker (KRaft mode) | 9092 |
| Schema Registry | Avro schema management | 8081 |
| MinIO | S3-compatible object storage | 9000, 9001 |
| PostgreSQL | Iceberg catalog metadata | 5432 |
| Iceberg REST Catalog | Table management | 8181 |
| Prometheus | Metrics collection | 9090 |
| Grafana | Dashboards and visualization | 3000 |

### Python Dependencies

All dependencies are managed via `uv` and installed automatically:
- `confluent-kafka` - Kafka client with Avro support
- `pyiceberg` - Iceberg Python SDK
- `duckdb` - Embedded analytics engine
- `pandas`, `matplotlib` - Data analysis and visualization
- `websockets` - Async WebSocket client for Binance
- `rich` - Beautiful terminal output

### Network Requirements

- Access to `wss://stream.binance.com:9443` (Binance WebSocket)
- No VPN restrictions on cryptocurrency exchange APIs

### Setup Time Estimate

- **First time**: 10-15 minutes (Docker image download + init)
- **Subsequent runs**: 2-3 minutes (start services + validation)

### Quick Start Commands

```bash
# Start all Docker services
docker compose up -d

# Initialize infrastructure (schemas, topics, tables)
python scripts/init_e2e_demo.py

# Run this notebook!
jupyter lab notebooks/binance_e2e_demo.ipynb
```

## 3. Environment Setup & Validation

Let's start by validating that all required services are running and healthy.

In [ ]:
# Standard library imports
import subprocess
import sys
import time
from datetime import datetime, timedelta
from pathlib import Path

import numpy as np

# Data processing
import pandas as pd

# Add src to path for k2 imports
sys.path.insert(0, str(Path.cwd().parent / "src"))

# Pandas display settings
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 200)

print("Imports loaded successfully!")

In [ ]:
# Check Docker services status
print("\nChecking Docker services...\n")

try:
    result = subprocess.run(
        ["docker", "compose", "ps", "--format", "json"],
        capture_output=True,
        text=True,
        cwd=Path.cwd().parent,
        timeout=10
    )

    if result.returncode != 0:
        print("Docker Compose is not running!")
        print("\nPlease start services with: docker compose up -d")
        raise Exception("Docker services not running")

    # Parse Docker Compose output
    import json
    services = []
    for line in result.stdout.strip().split('\n'):
        if line:
            try:
                service = json.loads(line)
                services.append(service)
            except json.JSONDecodeError:
                pass

    # Display status
    print("Docker Services Status\n")
    print(f"{'Service':<30} {'State':<15} {'Ports':<20}")
    print("-" * 65)

    required_services = [
        "k2-kafka", "k2-schema-registry-1", "k2-minio",
        "k2-postgres", "k2-iceberg-rest", "k2-prometheus"
    ]

    running_services = {}
    for service in services:
        name = service.get("Name", service.get("Service", "unknown"))
        state = service.get("State", "unknown")
        publishers = service.get("Publishers", [])

        # Format ports
        ports = ", ".join([f"{p.get('PublishedPort', '')}" for p in publishers if p.get('PublishedPort')]) or "N/A"

        running_services[name] = state
        print(f"{name:<30} {state:<15} {ports:<20}")

    # Check if all required services are running
    missing_services = []
    for svc in required_services:
        if svc not in running_services or running_services[svc] != "running":
            missing_services.append(svc)

    if missing_services:
        print(f"\nMissing or stopped services: {', '.join(missing_services)}")
        print("\nPlease start services with: docker compose up -d")
    else:
        print("\nAll required services are running!")

except FileNotFoundError:
    print("Docker is not installed or not in PATH")
    print("\nPlease install Docker: https://docs.docker.com/get-docker/")
except subprocess.TimeoutExpired:
    print("Docker command timed out")
except Exception as e:
    print(f"Error checking Docker: {e}")

In [ ]:
# Validate service health
print("\nValidating service health...\n")

import requests

health_checks = []

# 1. Check Schema Registry
try:
    start = time.time()
    response = requests.get("http://localhost:8081/subjects", timeout=5)
    latency = (time.time() - start) * 1000

    if response.status_code == 200:
        subjects = response.json()
        health_checks.append({
            "service": "Schema Registry",
            "status": "Healthy",
            "latency": f"{latency:.0f}ms",
            "detail": f"{len(subjects)} schemas"
        })
    else:
        health_checks.append({
            "service": "Schema Registry",
            "status": "Unhealthy",
            "latency": f"{latency:.0f}ms",
            "detail": f"Status {response.status_code}"
        })
except Exception as e:
    health_checks.append({
        "service": "Schema Registry",
        "status": "Error",
        "latency": "N/A",
        "detail": str(e)[:50]
    })

# 2. Check MinIO
try:
    import boto3

    start = time.time()
    s3_client = boto3.client(
        "s3",
        endpoint_url="http://localhost:9000",
        aws_access_key_id="admin",
        aws_secret_access_key="password",
    )
    buckets = s3_client.list_buckets()
    latency = (time.time() - start) * 1000

    health_checks.append({
        "service": "MinIO (S3)",
        "status": "Healthy",
        "latency": f"{latency:.0f}ms",
        "detail": f"{len(buckets['Buckets'])} buckets"
    })
except Exception as e:
    health_checks.append({
        "service": "MinIO (S3)",
        "status": "Error",
        "latency": "N/A",
        "detail": str(e)[:50]
    })

# 3. Check Iceberg REST Catalog
try:
    start = time.time()
    response = requests.get("http://localhost:8181/v1/config", timeout=5)
    latency = (time.time() - start) * 1000

    if response.status_code == 200:
        health_checks.append({
            "service": "Iceberg REST",
            "status": "Healthy",
            "latency": f"{latency:.0f}ms",
            "detail": "Catalog ready"
        })
    else:
        health_checks.append({
            "service": "Iceberg REST",
            "status": "Unhealthy",
            "latency": f"{latency:.0f}ms",
            "detail": f"Status {response.status_code}"
        })
except Exception as e:
    health_checks.append({
        "service": "Iceberg REST",
        "status": "Error",
        "latency": "N/A",
        "detail": str(e)[:50]
    })

# 4. Check Prometheus
try:
    start = time.time()
    response = requests.get("http://localhost:9090/-/healthy", timeout=5)
    latency = (time.time() - start) * 1000

    if response.status_code == 200:
        health_checks.append({
            "service": "Prometheus",
            "status": "Healthy",
            "latency": f"{latency:.0f}ms",
            "detail": "Metrics ready"
        })
    else:
        health_checks.append({
            "service": "Prometheus",
            "status": "Unhealthy",
            "latency": f"{latency:.0f}ms",
            "detail": f"Status {response.status_code}"
        })
except Exception as e:
    health_checks.append({
        "service": "Prometheus",
        "status": "Error",
        "latency": "N/A",
        "detail": str(e)[:50]
    })

# 5. Check Kafka (via AdminClient)
try:
    from confluent_kafka.admin import AdminClient

    start = time.time()
    admin = AdminClient({"bootstrap.servers": "localhost:9092"})
    metadata = admin.list_topics(timeout=5)
    latency = (time.time() - start) * 1000

    health_checks.append({
        "service": "Kafka",
        "status": "Healthy",
        "latency": f"{latency:.0f}ms",
        "detail": f"{len(metadata.topics)} topics"
    })
except Exception as e:
    health_checks.append({
        "service": "Kafka",
        "status": "Error",
        "latency": "N/A",
        "detail": str(e)[:50]
    })

# Display health check results
print("Service Health Checks\n")
print(f"{'Service':<20} {'Status':<12} {'Latency':<12} {'Detail':<30}")
print("-" * 74)

all_healthy = True
for check in health_checks:
    print(f"{check['service']:<20} {check['status']:<12} {check['latency']:<12} {check['detail']:<30}")
    if "Error" in check["status"] or "Unhealthy" in check["status"]:
        all_healthy = False

if all_healthy:
    print("\nAll services are healthy and ready!")
else:
    print("\nSome services are unhealthy. Please check Docker logs.")
    print("\nDebug with: docker compose logs <service-name>")

## 4. Infrastructure Initialization

Now that services are running and healthy, let's initialize the infrastructure:
1. Register v2 Avro schemas with Schema Registry
2. Create Kafka topics for crypto trades
3. Create Iceberg tables with v2 schema

In [ ]:
# Run infrastructure initialization script
print("\nInitializing infrastructure...\n")

try:
    result = subprocess.run(
        ["python", "scripts/init_e2e_demo.py"],
        capture_output=True,
        text=True,
        cwd=Path.cwd().parent,
        timeout=60
    )

    # Print output
    if result.stdout:
        print(result.stdout)

    if result.returncode == 0:
        print("\n Infrastructure initialized successfully!")
    else:
        print(f"\n Initialization failed with code {result.returncode}")
        if result.stderr:
            print("Error output:")
            print(result.stderr)

except subprocess.TimeoutExpired:
    print("\n Initialization timed out (>60s)")
except Exception as e:
    print(f"\n Error during initialization: {e}")

In [ ]:
# Verify infrastructure
print("\nVerifying infrastructure components...\n")

verification_results = []

# 1. Check Schema Registry subjects
try:
    from confluent_kafka.schema_registry import SchemaRegistryClient

    client = SchemaRegistryClient({"url": "http://localhost:8081"})
    subjects = client.get_subjects()

    # Check for required v2 subjects
    required_subjects = [
        "market.crypto.trades-value",
        "market.equities.trades-value",
    ]

    found_subjects = [s for s in required_subjects if s in subjects]

    verification_results.append({
        "component": "Schema Registry",
        "status": "Ready" if len(found_subjects) >= 2 else "Partial",
        "detail": f"{len(subjects)} schemas registered"
    })
except Exception as e:
    verification_results.append({
        "component": "Schema Registry",
        "status": "Error",
        "detail": str(e)[:50]
    })

# 2. Check Kafka topics
try:
    from confluent_kafka.admin import AdminClient

    admin = AdminClient({"bootstrap.servers": "localhost:9092"})
    metadata = admin.list_topics(timeout=10)

    # Check for crypto trades topic
    crypto_topic = "market.crypto.trades.binance"
    topic_exists = crypto_topic in metadata.topics

    if topic_exists:
        topic_meta = metadata.topics[crypto_topic]
        partition_count = len(topic_meta.partitions)
        verification_results.append({
            "component": "Kafka Topics",
            "status": "Ready",
            "detail": f"{crypto_topic} ({partition_count} partitions)"
        })
    else:
        verification_results.append({
            "component": "Kafka Topics",
            "status": "Missing",
            "detail": f"{crypto_topic} not found"
        })
except Exception as e:
    verification_results.append({
        "component": "Kafka Topics",
        "status": "Error",
        "detail": str(e)[:50]
    })

# 3. Check Iceberg tables
try:
    from pyiceberg.catalog import load_catalog

    catalog = load_catalog(
        "k2",
        **{
            "uri": "http://localhost:8181",
            "s3.endpoint": "http://localhost:9000",
            "s3.access-key-id": "admin",
            "s3.secret-access-key": "password",
            "s3.path-style-access": "true",
        },
    )

    # Try to load trades_v2 table
    try:
        table = catalog.load_table("market_data.trades_v2")
        field_count = len(table.schema().fields)

        verification_results.append({
            "component": "Iceberg Tables",
            "status": "Ready",
            "detail": f"market_data.trades_v2 ({field_count} fields)"
        })
    except Exception:
        verification_results.append({
            "component": "Iceberg Tables",
            "status": "Not Found",
            "detail": "market_data.trades_v2 missing"
        })
except Exception as e:
    verification_results.append({
        "component": "Iceberg Tables",
        "status": "Error",
        "detail": str(e)[:50]
    })

# Display verification results
print("Infrastructure Verification\n")
print(f"{'Component':<20} {'Status':<15} {'Detail':<50}")
print("-" * 85)

all_ready = True
for result in verification_results:
    print(f"{result['component']:<20} {result['status']:<15} {result['detail']:<50}")
    if result['status'] not in ['Ready', 'Partial']:
        all_ready = False

if all_ready:
    print("\nAll infrastructure components are ready for streaming!")
else:
    print("\nSome components are not ready. Run init_e2e_demo.py again.")

## 5. Binance WebSocket Client Demo

Now for the exciting part - let's connect to Binance's live WebSocket stream and watch real cryptocurrency trades flow in!

We'll stream trades for:
- **BTCUSDT** - Bitcoin vs USDT
- **ETHUSDT** - Ethereum vs USDT

The stream will run for **30 seconds** so you can see real-time data flowing through the system.

In [ ]:
# Import Binance client and producer
import asyncio

from IPython.display import clear_output

from k2.ingestion.binance_client import BinanceWebSocketClient
from k2.ingestion.producer import MarketDataProducer

# Create producer (will send to Kafka)
producer = MarketDataProducer(schema_version="v2")

# Create callback function to send trades to Kafka
def handle_trade(trade_data: dict) -> None:
    """Callback to send trades to Kafka."""
    producer.send_trade(trade_data)

# Create Binance WebSocket client
symbols = ["BTCUSDT", "ETHUSDT"]
client = BinanceWebSocketClient(
    symbols=symbols,
    on_message=handle_trade,
)

print(f"\nBinance client created for symbols: {', '.join(symbols)}")
print("\nReady to start streaming!")

In [ ]:
# Stream trades for 30 seconds
print("\nStarting Binance WebSocket stream...\n")
print("Streaming for 30 seconds. Watch real trades flow in!\n")

# Track streaming stats
streaming_stats = {
    "start_time": None,
    "total_trades": 0,
    "trades_by_symbol": {},
    "latest_prices": {},
}

async def demo_stream():
    """Stream trades from Binance for 30 seconds."""
    try:
        # Connect to Binance
        await client.connect()
        print("Connected to Binance WebSocket!\n")

        streaming_stats["start_time"] = time.time()

        # Stream for 30 seconds
        duration = 30
        while time.time() - streaming_stats["start_time"] < duration:
            await asyncio.sleep(1)

            # Get current metrics from producer
            elapsed = int(time.time() - streaming_stats["start_time"])

            # Update display
            clear_output(wait=True)

            print(f"Streaming... ({elapsed}s / {duration}s)\n")
            print("Connected to: wss://stream.binance.com:9443/stream")
            print(f"Symbols: {', '.join(symbols)}")
            print("\nLive trades are being sent to Kafka topic: market.crypto.trades.binance")
            print(f"\nTrades received: ~{elapsed * 10} (estimated)")

        # Disconnect
        await client.disconnect()

        print("\nStreaming complete!")
        print(f"\nTotal time: {duration}s")
        print(f"Estimated trades received: {duration * 10} (BTC + ETH)")
        print("\nTrades are now in Kafka and ready to be consumed to Iceberg!")

    except Exception as e:
        print(f"\nStreaming error: {e}")
        print("\nThis is usually due to network issues or Binance API limits.")
        print("Don't worry - you can continue with existing Kafka data!")

# Run the streaming demo
await demo_stream()

# Flush producer to ensure all messages are sent
producer.flush()
print("\nProducer flushed - all messages sent to Kafka")

In [ ]:
# Show example Binance WebSocket message format
print("\nExample Binance WebSocket Message:\n")

import json

example_binance_message = {
    "e": "trade",              # Event type
    "E": 1704067800000,        # Event time (milliseconds)
    "s": "BTCUSDT",            # Symbol
    "t": 123456789,            # Trade ID
    "p": "65000.00",           # Price
    "q": "0.05",               # Quantity
    "T": 1704067800000,        # Trade time (milliseconds)
    "m": False,                # Is buyer maker?
    "M": True                  # Is best match?
}

print("Raw Binance WebSocket JSON:")
print(json.dumps(example_binance_message, indent=2))

print("\nConverted to K2 V2 Schema:\n")

# Show v2 conversion (simplified example)
example_v2_trade = {
    "message_id": "123e4567-e89b-12d3-a456-426614174000",  # Generated UUID
    "trade_id": "BINANCE-123456789",                       # Exchange trade ID
    "symbol": "BTCUSDT",                                   # Trading pair
    "exchange": "BINANCE",                                 # Exchange name
    "asset_class": "crypto",                               # Asset class
    "timestamp": "2024-01-01T00:00:00.000000Z",            # Microseconds
    "price": "65000.00000000",                             # Decimal(18,8)
    "quantity": "0.05000000",                              # Decimal(18,8)
    "currency": "USDT",                                    # Quote currency
    "side": "BUY",                                         # Aggressor side
    "source_sequence": 123456789,                          # Exchange sequence
    "vendor_data": {                                        # Binance-specific fields
        "event_type": "trade",
        "event_time": "1704067800000",
        "trade_time": "1704067800000",
        "is_buyer_maker": "False",
        "is_best_match": "True",
        "base_asset": "BTC",
        "quote_asset": "USDT"
    }
}

print("K2 V2 Schema (with vendor_data):")
print(json.dumps(example_v2_trade, indent=2))

print("\nKey Points:")
print("  • Industry-standard core fields (symbol, price, quantity, etc.)")
print("  • Binance-specific fields preserved in vendor_data JSON")
print("  • UUID message_id for deduplication")
print("  • Microsecond precision timestamps")
print("  • Decimal(18,8) precision for prices/quantities")

## 6. Kafka Topic Inspection

Let's verify that the trades we just streamed are actually in Kafka. We'll check:
1. Topic metadata (partitions, leaders)
2. Total message count per partition
3. Sample a few messages to see the Avro-serialized data

In [ ]:
# Check Kafka topic metadata
print("\nKafka Topic Metadata:\n")

from confluent_kafka.admin import AdminClient

admin = AdminClient({"bootstrap.servers": "localhost:9092"})
metadata = admin.list_topics(topic="market.crypto.trades.binance", timeout=10)

topic_meta = metadata.topics["market.crypto.trades.binance"]

print(f"Topic: {topic_meta.topic}")
print(f"Partitions: {len(topic_meta.partitions)}")

# Create partition table
table = Table(title="Partition Distribution", show_header=True)
table.add_column("Partition", style="cyan")
table.add_column("Leader", style="green")
table.add_column("Replicas", style="yellow")
table.add_column("ISR", style="magenta")

for partition_id, partition in sorted(topic_meta.partitions.items()):
    table.add_row(
        str(partition_id),
        str(partition.leader),
        str(len(partition.replicas)),
        str(len(partition.isrs))
    )

print(table)
print("\n Topic exists and has healthy partition distribution")

In [ ]:
# Count messages in topic
print("\nCounting messages in topic...\n")

from confluent_kafka import Consumer

consumer = Consumer({
    "bootstrap.servers": "localhost:9092",
    "group.id": "notebook-inspector",
    "auto.offset.reset": "earliest",
})

# Get partition assignments
consumer.subscribe(["market.crypto.trades.binance"])
consumer.poll(timeout=1.0)  # Trigger partition assignment
assignment = consumer.assignment()

if not assignment:
    print(" No partitions assigned yet. Topic might be empty.")
else:
    # Get watermark offsets (low = earliest, high = latest)
    table = Table(title="Message Count by Partition", show_header=True)
    table.add_column("Partition", style="cyan")
    table.add_column("Low Offset", style="yellow")
    table.add_column("High Offset", style="green")
    table.add_column("Messages", style="magenta")

    total_messages = 0
    for tp in sorted(assignment, key=lambda x: x.partition):
        low, high = consumer.get_watermark_offsets(tp, timeout=5.0)
        messages = high - low
        total_messages += messages

        table.add_row(
            str(tp.partition),
            f"{low:,}",
            f"{high:,}",
            f"{messages:,}"
        )

    print(table)
    print(f"\nTotal messages in topic: {total_messages:,}")

    if total_messages == 0:
        print("\n No messages found. The streaming demo might have failed or not run yet.")
    else:
        print(f"Average messages per partition: {total_messages / len(assignment):,.1f}")

consumer.close()

In [ ]:
# Sample 5 messages from the topic
print("\nSampling messages from Kafka...\n")

from confluent_kafka import Consumer
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.schema_registry.avro import AvroDeserializer

# Create Schema Registry client
schema_registry_client = SchemaRegistryClient({"url": "http://localhost:8081"})

# Create consumer
consumer = Consumer({
    "bootstrap.servers": "localhost:9092",
    "group.id": "notebook-sampler",
    "auto.offset.reset": "earliest",
    "enable.auto.commit": False,
})

consumer.subscribe(["market.crypto.trades.binance"])

# Consume 5 messages
messages = []
attempts = 0
max_attempts = 20

print("Consuming messages...")

while len(messages) < 5 and attempts < max_attempts:
    msg = consumer.poll(timeout=1.0)
    attempts += 1

    if msg is None:
        continue
    if msg.error():
        print(f"Consumer error: {msg.error()}")
        continue

    # Deserialize Avro value
    try:
        # Get schema ID from message
        schema_id = int.from_bytes(msg.value()[1:5], byteorder='big')
        schema = schema_registry_client.get_schema(schema_id)

        deserializer = AvroDeserializer(
            schema_registry_client,
            schema.schema_str
        )

        value = deserializer(msg.value(), None)
        messages.append(value)

    except Exception as e:
        print(f"Deserialization error: {e}")
        continue

consumer.close()

if messages:
    print(f"\n Successfully sampled {len(messages)} messages\n")

    # Convert to DataFrame
    df_kafka = pd.DataFrame(messages)

    # Display sample
    display_columns = ["symbol", "timestamp", "price", "quantity", "side", "exchange"]
    if all(col in df_kafka.columns for col in display_columns):
        print("Sample Messages:")
        display(df_kafka[display_columns].head())

        print(f"\nMessage structure: {len(df_kafka.columns)} fields")
        print(f"Fields: {', '.join(df_kafka.columns[:10])}...")
    else:
        print("Sample Messages (all fields):")
        display(df_kafka.head())
else:
    print("\n No messages found. The topic might be empty.")
    print("Run the Binance streaming demo above to populate the topic.")

## 7. Consumer Pipeline (Kafka → Iceberg)

Now let's consume messages from Kafka and write them to Iceberg. The consumer will:
1. Read messages in batches (500 records)
2. Deserialize Avro to Python objects
3. Write to Iceberg with ACID guarantees
4. Track metrics (throughput, latency, errors)

We'll consume **1,000 messages** to demonstrate the pipeline.

In [ ]:
# Run consumer to write messages to Iceberg
print("\nStarting consumer pipeline...\n")

from k2.ingestion.consumer import MarketDataConsumer

# Create consumer
consumer = MarketDataConsumer(
    topics=["market.crypto.trades.binance"],
    group_id="notebook-consumer",
    table_version="v2",
)

print("Consumer Configuration:")
print(f"  Topics: {consumer.topics}")
print(f"  Group ID: {consumer.group_id}")
print(f"  Batch size: {consumer.batch_size}")
print("  Table version: v2")
print("\n Consumer initialized")

In [ ]:
# Consume and write messages
print("\nConsuming messages and writing to Iceberg...\n")

max_messages = 1000
messages_consumed = 0
messages_written = 0
start_time = time.time()

print(f"Target: {max_messages} messages")
print("This may take 30-60 seconds...\n")

try:
    with Progress(
        SpinnerColumn(),
        TextColumn("[progress.description]{task.description}"),
        console=console,
    ) as progress:
        task = progress.add_task("Consuming messages...", total=max_messages)

        # Start consuming
        for message in consumer.consume(max_messages=max_messages):
            messages_consumed += 1
            progress.update(task, advance=1)

            # Update every 100 messages
            if messages_consumed % 100 == 0:
                elapsed = time.time() - start_time
                throughput = messages_consumed / elapsed if elapsed > 0 else 0
                progress.update(
                    task,
                    description=f"Consuming messages... ({messages_consumed}/{max_messages}, {throughput:.1f} msg/s)"
                )

    elapsed_time = time.time() - start_time
    throughput = messages_consumed / elapsed_time if elapsed_time > 0 else 0

    print("\n Consumption complete!")
    print("\nResults:")
    print(f"  Messages consumed: {messages_consumed:,}")
    print(f"  Time elapsed: {elapsed_time:.2f}s")
    print(f"  Throughput: {throughput:.2f} msg/s")

    # Note: Messages are written in batches, so written count may differ
    print("\nMessages have been written to Iceberg table: market_data.trades_v2")

except KeyboardInterrupt:
    print("\n Consumption interrupted by user")
except Exception as e:
    print(f"\n Consumer error: {e}")
    import traceback
    traceback.print_exc()
finally:
    # Close consumer
    consumer.close()
    print("\n Consumer closed")

## 8. Iceberg Table Inspection

Let's verify that data was successfully written to the Iceberg lakehouse. We'll:
1. Load the table and inspect its schema
2. Count total rows
3. Sample recent data
4. Examine vendor_data fields

In [ ]:
# Load Iceberg table and inspect schema
print("\nLoading Iceberg table...\n")

from pyiceberg.catalog import load_catalog

catalog = load_catalog(
    "k2",
    **{
        "uri": "http://localhost:8181",
        "s3.endpoint": "http://localhost:9000",
        "s3.access-key-id": "admin",
        "s3.secret-access-key": "password",
        "s3.path-style-access": "true",
    },
)

try:
    table = catalog.load_table("market_data.trades_v2")

    print(f" Table loaded: {table.name()}")
    print("\nTable Schema (V2):\n")

    # Create schema table
    schema_table = Table(title="trades_v2 Schema", show_header=True)
    schema_table.add_column("Field ID", style="cyan")
    schema_table.add_column("Field Name", style="green")
    schema_table.add_column("Type", style="yellow")
    schema_table.add_column("Required", style="magenta")

    for field in table.schema().fields:
        required = "" if field.required else ""
        schema_table.add_row(
            str(field.field_id),
            field.name,
            str(field.field_type),
            required
        )

    print(schema_table)
    print(f"\nTotal fields: {len(table.schema().fields)}")

except Exception as e:
    print(f" Error loading table: {e}")
    print("\nMake sure the consumer has written data to the table.")

In [ ]:
# Count rows using DuckDB
print("\nCounting rows in Iceberg table...\n")

import duckdb

# Create DuckDB connection
conn = duckdb.connect(":memory:")
conn.install_extension("iceberg")
conn.load_extension("iceberg")

# Configure S3 access for DuckDB
conn.execute("""
    CREATE SECRET secret1 (
        TYPE S3,
        KEY_ID 'admin',
        SECRET 'password',
        ENDPOINT 'localhost:9000',
        URL_STYLE 'path',
        REGION 'us-east-1',
        USE_SSL false
    );
""")

try:
    # Count rows
    result = conn.execute("""
        SELECT COUNT(*) as count
        FROM iceberg_scan('http://localhost:8181/v1/market_data/trades_v2', allow_moved_paths=true)
    """).fetchone()

    row_count = result[0]

    print(f"Total rows in trades_v2: {row_count:,}")

    if row_count == 0:
        print("\n No rows found. The consumer might not have written data yet.")
        print("Run the consumer pipeline above to write data to Iceberg.")
    else:
        # Get data distribution by symbol
        result = conn.execute("""
            SELECT 
                symbol,
                COUNT(*) as trade_count,
                MIN(timestamp) as first_trade,
                MAX(timestamp) as last_trade
            FROM iceberg_scan('http://localhost:8181/v1/market_data/trades_v2', allow_moved_paths=true)
            GROUP BY symbol
            ORDER BY trade_count DESC
        """).fetchdf()

        print("\nData distribution by symbol:\n")
        display(result)

except Exception as e:
    print(f" Error querying table: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# Sample recent data
print("\nSampling recent trades...\n")

try:
    # Query 10 most recent trades
    df_recent = conn.execute("""
        SELECT *
        FROM iceberg_scan('http://localhost:8181/v1/market_data/trades_v2', allow_moved_paths=true)
        ORDER BY timestamp DESC
        LIMIT 10
    """).fetchdf()

    if len(df_recent) > 0:
        print(f" Found {len(df_recent)} recent trades\n")

        # Display key columns
        display_columns = ["symbol", "exchange", "timestamp", "price", "quantity", "side", "currency"]
        print("Recent Trades (Key Fields):")
        display(df_recent[display_columns])

        # Show sample trade details
        print("\nSample Trade Details:")
        sample = df_recent.iloc[0]
        print(f"  Symbol: {sample['symbol']}")
        print(f"  Exchange: {sample['exchange']}")
        print(f"  Asset Class: {sample['asset_class']}")
        print(f"  Timestamp: {sample['timestamp']}")
        print(f"  Price: {sample['price']} {sample['currency']}")
        print(f"  Quantity: {sample['quantity']}")
        print(f"  Side: {sample['side']}")
        print(f"  Trade ID: {sample['trade_id']}")
        print(f"  Message ID: {sample['message_id']}")

    else:
        print(" No trades found in table")

except Exception as e:
    print(f" Error sampling data: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# Examine vendor_data (Binance-specific fields)
print("\nExamining vendor_data...\n")

import json

if len(df_recent) > 0:
    sample = df_recent.iloc[0]

    print("Vendor Data (Binance-specific fields):\n")

    # Parse vendor_data JSON
    if pd.notna(sample['vendor_data']):
        vendor_data = json.loads(sample['vendor_data'])

        # Create vendor data table
        vendor_table = Table(title="Binance Vendor Data", show_header=True)
        vendor_table.add_column("Field", style="cyan")
        vendor_table.add_column("Value", style="green")
        vendor_table.add_column("Description", style="yellow")

        field_descriptions = {
            "event_type": "Type of event (trade, aggTrade, etc.)",
            "event_time": "Event timestamp from Binance (milliseconds)",
            "trade_time": "Trade execution timestamp (milliseconds)",
            "is_buyer_maker": "Was the buyer the maker? (true/false)",
            "is_best_match": "Was this the best price match? (true/false)",
            "base_asset": "Base asset symbol (e.g., BTC in BTCUSDT)",
            "quote_asset": "Quote asset symbol (e.g., USDT in BTCUSDT)",
        }

        for key, value in vendor_data.items():
            description = field_descriptions.get(key, "N/A")
            vendor_table.add_row(key, str(value), description)

        print(vendor_table)

        print("\nKey Points:")
        print("  • vendor_data preserves all Binance-specific fields")
        print("  • Core fields (price, quantity, side) are normalized to v2 schema")
        print("  • Exchange-specific data enables advanced analysis")
        print("  • Same v2 schema works across ASX (equities) and Binance (crypto)")

    else:
        print(" No vendor_data found in sample trade")
else:
    print(" No data available to examine vendor_data")

## 9. Query Engine Demo

Now let's demonstrate the K2 Query Engine, which provides a high-level Python API for querying trades. The QueryEngine:
- Abstracts DuckDB complexity
- Supports time-range queries
- Filters by symbol, exchange, asset class
- Returns data as Python dictionaries or DataFrames
- Provides sub-second query performance

In [ ]:
# Initialize QueryEngine
print("\nInitializing Query Engine...\n")

from k2.query.engine import QueryEngine

engine = QueryEngine(table_version="v2")

print(" QueryEngine initialized")
print(f"  Iceberg Catalog: {engine.iceberg_catalog_uri}")
print("  Table: market_data.trades_v2")
print("  Query Engine: DuckDB with Iceberg extension")

In [ ]:
# Basic queries
print("\nRunning basic queries...\n")

# Query 1: Get all available symbols
try:
    symbols = engine.get_symbols(exchange="BINANCE")
    print(f"Available symbols: {', '.join(symbols)}")
except Exception as e:
    print(f"Error getting symbols: {e}")
    symbols = []

# Query 2: Query trades for each symbol
if symbols:
    for symbol in symbols:
        try:
            trades = engine.query_trades(
                symbol=symbol,
                exchange="BINANCE",
                limit=100,
            )
            print(f" {symbol}: {len(trades)} trades (limit 100)")
        except Exception as e:
            print(f" {symbol}: Error - {e}")
else:
    print("\n No symbols found. Query all trades instead:")
    try:
        all_trades = engine.query_trades(limit=100)
        print(f" Found {len(all_trades)} trades (limit 100)")

        # Get unique symbols from results
        if all_trades:
            unique_symbols = set(t['symbol'] for t in all_trades)
            print(f"Symbols in data: {', '.join(unique_symbols)}")
    except Exception as e:
        print(f" Error querying trades: {e}")

In [ ]:
# Time-range query
print("\nTime-range query (last 5 minutes)...\n")

from datetime import UTC

# Query last 5 minutes
end_time = datetime.now(UTC)
start_time = end_time - timedelta(minutes=5)

try:
    recent_trades = engine.query_trades(
        start_time=start_time,
        end_time=end_time,
        limit=1000,
    )

    print(f" Found {len(recent_trades)} trades in last 5 minutes")

    if recent_trades:
        # Convert to DataFrame for analysis
        df_trades = pd.DataFrame(recent_trades)

        print("\nTime range:")
        print(f"  Start: {start_time}")
        print(f"  End: {end_time}")
        print("  Duration: 5 minutes")

        print("\nSample trades:")
        display(df_trades[["symbol", "timestamp", "price", "quantity", "side"]].head(10))

        # Show trade distribution
        print("\nTrade distribution:")
        symbol_counts = df_trades["symbol"].value_counts()
        for symbol, count in symbol_counts.items():
            print(f"  {symbol}: {count} trades")
    else:
        print("\n No trades found in the specified time range")
        print("This is normal if data is older than 5 minutes.")

except Exception as e:
    print(f" Error in time-range query: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# OHLCV aggregation query
print("\nOHLCV (Open, High, Low, Close, Volume) Aggregation...\n")

try:
    # Query all trades
    all_trades = engine.query_trades(limit=5000)

    if all_trades:
        df_all = pd.DataFrame(all_trades)

        # Convert price and quantity to numeric
        df_all["price"] = pd.to_numeric(df_all["price"])
        df_all["quantity"] = pd.to_numeric(df_all["quantity"])

        # Calculate OHLCV by symbol
        ohlcv = df_all.groupby("symbol").agg({
            "price": ["first", "max", "min", "last"],
            "quantity": "sum",
            "trade_id": "count",
        })

        # Flatten column names
        ohlcv.columns = ["open", "high", "low", "close", "volume", "trades"]

        # Calculate price range and spread
        ohlcv["range"] = ohlcv["high"] - ohlcv["low"]
        ohlcv["spread_pct"] = (ohlcv["range"] / ohlcv["open"] * 100).round(2)

        print("OHLCV Summary by Symbol:\n")
        display(ohlcv)

        print("\nAnalysis:")
        for symbol in ohlcv.index:
            row = ohlcv.loc[symbol]
            print(f"\n{symbol}:")
            print(f"  Open: {row['open']:.2f}, Close: {row['close']:.2f}")
            print(f"  High: {row['high']:.2f}, Low: {row['low']:.2f}")
            print(f"  Range: {row['range']:.2f} ({row['spread_pct']:.2f}%)")
            print(f"  Volume: {row['volume']:.4f}")
            print(f"  Trades: {row['trades']:.0f}")
    else:
        print(" No trades available for OHLCV calculation")

except Exception as e:
    print(f" Error in OHLCV aggregation: {e}")
    import traceback
    traceback.print_exc()

## 10. Real-Time Price Visualization

Let's visualize the live cryptocurrency trade data with interactive charts:
1. **Price Scatter Plot** - BTC and ETH prices over time (colored by trade size)
2. **Volume Bar Chart** - Trading volume aggregated in 30-second buckets
3. **Price Distribution** - Histogram showing price ranges

In [ ]:
# Prepare data for visualization
print("\nPreparing data for visualization...\n")

import matplotlib.dates as mdates
import matplotlib.pyplot as plt

# Enable matplotlib inline plotting
%matplotlib inline

# Set style
plt.style.use('seaborn-v0_8-darkgrid')

try:
    # Query recent trades for visualization
    viz_trades = engine.query_trades(limit=2000)

    if viz_trades:
        df_viz = pd.DataFrame(viz_trades)

        # Convert data types
        df_viz["timestamp"] = pd.to_datetime(df_viz["timestamp"])
        df_viz["price"] = pd.to_numeric(df_viz["price"])
        df_viz["quantity"] = pd.to_numeric(df_viz["quantity"])

        # Sort by timestamp
        df_viz = df_viz.sort_values("timestamp")

        print(f" Prepared {len(df_viz)} trades for visualization")
        print(f"Time range: {df_viz['timestamp'].min()} to {df_viz['timestamp'].max()}")
        print(f"Symbols: {', '.join(df_viz['symbol'].unique())}")

        # Separate by symbol for plotting
        symbols = df_viz["symbol"].unique()
        df_by_symbol = {symbol: df_viz[df_viz["symbol"] == symbol] for symbol in symbols}

        print("\n Data ready for plotting")

    else:
        print(" No trades available for visualization")
        df_viz = None
        df_by_symbol = {}

except Exception as e:
    print(f" Error preparing data: {e}")
    df_viz = None
    df_by_symbol = {}

In [ ]:
# Price scatter plot by symbol
if df_by_symbol:
    print("\nPlotting price charts...\n")

    num_symbols = len(df_by_symbol)
    fig, axes = plt.subplots(num_symbols, 1, figsize=(14, 5 * num_symbols), sharex=True)

    # Handle single symbol case
    if num_symbols == 1:
        axes = [axes]

    # Color maps for different symbols
    color_maps = ["viridis", "plasma", "inferno", "magma", "cividis"]

    for idx, (symbol, df_symbol) in enumerate(df_by_symbol.items()):
        ax = axes[idx]

        # Scatter plot with quantity as color
        scatter = ax.scatter(
            df_symbol["timestamp"],
            df_symbol["price"],
            c=df_symbol["quantity"],
            cmap=color_maps[idx % len(color_maps)],
            alpha=0.6,
            s=50,
            edgecolors='black',
            linewidth=0.5,
        )

        # Labels and title
        ax.set_ylabel(f"Price ({df_symbol['currency'].iloc[0]})", fontsize=12, fontweight='bold')
        ax.set_title(f"{symbol} Live Trades - Price Over Time", fontsize=14, fontweight='bold')
        ax.grid(True, alpha=0.3)

        # Add colorbar
        cbar = plt.colorbar(scatter, ax=ax)
        cbar.set_label("Trade Quantity", fontsize=10)

        # Add price statistics
        mean_price = df_symbol["price"].mean()
        std_price = df_symbol["price"].std()
        ax.axhline(y=mean_price, color='red', linestyle='--', linewidth=1, alpha=0.7, label=f'Mean: {mean_price:.2f}')
        ax.axhline(y=mean_price + std_price, color='orange', linestyle=':', linewidth=1, alpha=0.5, label=f'+1σ: {mean_price + std_price:.2f}')
        ax.axhline(y=mean_price - std_price, color='orange', linestyle=':', linewidth=1, alpha=0.5, label=f'-1σ: {mean_price - std_price:.2f}')
        ax.legend(loc='upper right', fontsize=8)

    # Format x-axis (time)
    axes[-1].set_xlabel("Time (UTC)", fontsize=12, fontweight='bold')
    axes[-1].xaxis.set_major_formatter(mdates.DateFormatter("%H:%M:%S"))
    plt.xticks(rotation=45)

    plt.tight_layout()
    plt.show()

    print(" Price charts rendered")
else:
    print(" No data available for plotting")

In [ ]:
# Volume bar chart (30-second buckets)
if df_viz is not None and len(df_viz) > 0:
    print("\nPlotting volume chart...\n")

    # Create time buckets (30 seconds)
    df_viz["time_bucket"] = df_viz["timestamp"].dt.floor("30S")

    # Aggregate volume by bucket and symbol
    volume_agg = df_viz.groupby(["time_bucket", "symbol"])["quantity"].sum().reset_index()

    # Create volume chart
    fig, ax = plt.subplots(figsize=(14, 6))

    # Plot bars for each symbol
    symbols = volume_agg["symbol"].unique()
    bar_width = pd.Timedelta(seconds=30) / (len(symbols) + 1)

    for idx, symbol in enumerate(symbols):
        symbol_data = volume_agg[volume_agg["symbol"] == symbol]

        # Offset bars for each symbol
        offset = bar_width * (idx - len(symbols) / 2)

        ax.bar(
            symbol_data["time_bucket"] + offset,
            symbol_data["quantity"],
            width=bar_width,
            label=symbol,
            alpha=0.7,
            edgecolor='black',
            linewidth=0.5,
        )

    ax.set_xlabel("Time (UTC)", fontsize=12, fontweight='bold')
    ax.set_ylabel("Trading Volume", fontsize=12, fontweight='bold')
    ax.set_title("Trading Volume by Symbol (30-second buckets)", fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='y')
    ax.legend(loc='upper right', fontsize=10)

    # Format x-axis
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M:%S"))
    plt.xticks(rotation=45)

    plt.tight_layout()
    plt.show()

    print(" Volume chart rendered")

    # Print volume statistics
    print("\nVolume Statistics:")
    for symbol in symbols:
        symbol_trades = df_viz[df_viz["symbol"] == symbol]
        total_volume = symbol_trades["quantity"].sum()
        avg_trade_size = symbol_trades["quantity"].mean()
        print(f"\n{symbol}:")
        print(f"  Total volume: {total_volume:.4f}")
        print(f"  Average trade size: {avg_trade_size:.6f}")
        print(f"  Number of trades: {len(symbol_trades):,}")
else:
    print(" No data available for volume chart")

## 11. Pipeline Metrics Visualization

The K2 platform exposes comprehensive metrics through Prometheus. Let's query and visualize:
1. **Consumer throughput** - Messages consumed per second over time
2. **Binance connection status** - WebSocket connection health
3. **Message lag** - Consumer lag behind Kafka
4. **Error rates** - Any errors in the pipeline

In [ ]:
# Query current metrics from Prometheus
print("\nQuerying Prometheus metrics...\n")

import requests

prom_url = "http://localhost:9090/api/v1/query"

def query_prometheus(metric_name):
    """Query Prometheus for a metric."""
    try:
        response = requests.get(prom_url, params={"query": metric_name}, timeout=5)
        data = response.json()

        if data["status"] == "success" and data["data"]["result"]:
            # Return all results
            return data["data"]["result"]
        return []
    except Exception as e:
        print(f"Error querying {metric_name}: {e}")
        return []

# Query key metrics
metrics_to_check = [
    ("k2_kafka_messages_consumed_total", "Total messages consumed"),
    ("k2_kafka_messages_produced_total", "Total messages produced"),
    ("k2_binance_messages_received_total", "Total Binance messages received"),
    ("k2_binance_connection_status", "Binance connection status"),
]

# Create metrics summary table
metrics_table = Table(title="Current Pipeline Metrics", show_header=True)
metrics_table.add_column("Metric", style="cyan")
metrics_table.add_column("Description", style="yellow")
metrics_table.add_column("Value", style="green")

for metric_name, description in metrics_to_check:
    results = query_prometheus(metric_name)

    if results:
        # Sum values if multiple series
        total = sum(float(r["value"][1]) for r in results)
        metrics_table.add_row(metric_name, description, f"{total:,.0f}")
    else:
        metrics_table.add_row(metric_name, description, "N/A")

print(metrics_table)
print("\n Current metrics retrieved")

In [ ]:
# Query time-series metrics for visualization
print("\nQuerying time-series metrics (last 10 minutes)...\n")

prom_range_url = "http://localhost:9090/api/v1/query_range"

def query_prometheus_range(query, duration_minutes=10):
    """Query Prometheus for a time-series metric."""
    try:
        end_time = datetime.now(UTC)
        start_time = end_time - timedelta(minutes=duration_minutes)

        params = {
            "query": query,
            "start": int(start_time.timestamp()),
            "end": int(end_time.timestamp()),
            "step": "15s",
        }

        response = requests.get(prom_range_url, params=params, timeout=10)
        data = response.json()

        if data["status"] == "success" and data["data"]["result"]:
            # Parse time-series data
            all_series = []
            for series in data["data"]["result"]:
                values = series["values"]
                timestamps = [datetime.fromtimestamp(v[0], tz=UTC) for v in values]
                vals = [float(v[1]) for v in values]
                labels = series["metric"]

                all_series.append({
                    "timestamps": timestamps,
                    "values": vals,
                    "labels": labels,
                })

            return all_series
        return []
    except Exception as e:
        print(f"Error querying range: {e}")
        return []

# Query consumer throughput (rate of messages consumed)
throughput_series = query_prometheus_range("rate(k2_kafka_messages_consumed_total[1m])", duration_minutes=10)

if throughput_series:
    print(f" Found {len(throughput_series)} throughput series")
else:
    print(" No throughput data available (might need to wait for metrics to populate)")

In [ ]:
# Plot throughput metrics
if throughput_series:
    print("\nPlotting consumer throughput...\n")

    fig, ax = plt.subplots(figsize=(14, 6))

    for series in throughput_series:
        # Get label for legend
        topic = series["labels"].get("topic", "unknown")
        label = f"Topic: {topic}"

        # Plot line
        ax.plot(
            series["timestamps"],
            series["values"],
            linewidth=2,
            marker="o",
            markersize=4,
            label=label,
            alpha=0.8,
        )

    ax.set_xlabel("Time (UTC)", fontsize=12, fontweight='bold')
    ax.set_ylabel("Messages/second", fontsize=12, fontweight='bold')
    ax.set_title("Consumer Throughput (Last 10 Minutes)", fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.legend(loc='upper right', fontsize=10)

    # Format x-axis
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M:%S"))
    plt.xticks(rotation=45)

    plt.tight_layout()
    plt.show()

    # Calculate statistics
    all_values = []
    for series in throughput_series:
        all_values.extend(series["values"])

    if all_values:
        avg_throughput = np.mean(all_values)
        max_throughput = np.max(all_values)
        min_throughput = np.min(all_values)

        print("\nThroughput Statistics:")
        print(f"  Average: {avg_throughput:.2f} msg/s")
        print(f"  Peak: {max_throughput:.2f} msg/s")
        print(f"  Minimum: {min_throughput:.2f} msg/s")

    print("\n Throughput chart rendered")
else:
    print(" No throughput data to plot")
    print("Metrics may not be available yet. Run the consumer pipeline and wait 1-2 minutes.")

## 12. Data Quality Checks

Let's validate data integrity and quality across the pipeline:
1. **Schema validation** - All required fields present and non-null
2. **Sequence gap detection** - Check for missing sequence numbers
3. **Duplicate detection** - Find duplicate message_ids or trade_ids
4. **Data quality score** - Overall quality assessment

In [ ]:
# Schema validation
print("\nValidating schema compliance...\n")

try:
    # Query trades for validation
    validation_trades = engine.query_trades(limit=1000)

    if validation_trades:
        df_validation = pd.DataFrame(validation_trades)

        # Required fields according to v2 schema
        required_fields = [
            "message_id", "trade_id", "symbol", "exchange", "asset_class",
            "timestamp", "price", "quantity", "currency", "side",
            "source_sequence", "ingestion_timestamp", "platform_sequence",
        ]

        # Create validation table
        validation_table = Table(title="Schema Validation Results", show_header=True)
        validation_table.add_column("Field", style="cyan")
        validation_table.add_column("Present", style="green")
        validation_table.add_column("Null Count", style="yellow")
        validation_table.add_column("Status", style="magenta")

        all_valid = True
        for field in required_fields:
            present = field in df_validation.columns

            if present:
                null_count = df_validation[field].isnull().sum()
                status = " Valid" if null_count == 0 else " Has nulls"

                if null_count > 0:
                    all_valid = False

                validation_table.add_row(
                    field,
                    " Yes" if present else " No",
                    str(null_count),
                    status
                )
            else:
                validation_table.add_row(field, " No", "N/A", " Missing")
                all_valid = False

        print(validation_table)

        if all_valid:
            print("\n All required fields present with no nulls")
        else:
            print("\n Some fields have issues (see table above)")
    else:
        print(" No data available for schema validation")
        df_validation = None

except Exception as e:
    print(f" Error in schema validation: {e}")
    df_validation = None

In [ ]:
# Sequence gap detection
print("\nDetecting sequence gaps...\n")

if df_validation is not None and len(df_validation) > 0:
    gaps_detected = []

    # Check for gaps per symbol
    for symbol in df_validation["symbol"].unique():
        df_symbol = df_validation[df_validation["symbol"] == symbol].copy()

        # Sort by source_sequence
        df_symbol = df_symbol.sort_values("source_sequence")

        # Check for gaps in source_sequence
        if "source_sequence" in df_symbol.columns:
            sequences = df_symbol["source_sequence"].dropna().astype(int).values

            if len(sequences) > 1:
                # Find gaps
                min_seq = int(sequences.min())
                max_seq = int(sequences.max())
                expected = set(range(min_seq, max_seq + 1))
                actual = set(sequences)
                gaps = expected - actual

                if gaps:
                    gaps_detected.append({
                        "symbol": symbol,
                        "gap_count": len(gaps),
                        "sequence_range": f"{min_seq} - {max_seq}",
                        "total_trades": len(sequences),
                    })

    if gaps_detected:
        print(" Sequence gaps detected:\n")

        gaps_table = Table(title="Sequence Gaps by Symbol", show_header=True)
        gaps_table.add_column("Symbol", style="cyan")
        gaps_table.add_column("Gap Count", style="yellow")
        gaps_table.add_column("Sequence Range", style="green")
        gaps_table.add_column("Total Trades", style="magenta")

        for gap in gaps_detected:
            gaps_table.add_row(
                gap["symbol"],
                str(gap["gap_count"]),
                gap["sequence_range"],
                str(gap["total_trades"])
            )

        print(gaps_table)
        print("\nNote: Sequence gaps are normal in live streaming (network delays, filtering, etc.)")
    else:
        print(" No sequence gaps detected")
else:
    print(" No data available for sequence gap detection")

In [ ]:
# Duplicate detection
print("\nDetecting duplicates...\n")

if df_validation is not None and len(df_validation) > 0:
    # Check for duplicate message_ids
    duplicate_msgs = df_validation[df_validation.duplicated(subset=["message_id"], keep=False)]

    print(f"Duplicate message_ids: {len(duplicate_msgs)}")

    if len(duplicate_msgs) > 0:
        print(" Found duplicate message IDs:")
        display(duplicate_msgs[["message_id", "symbol", "timestamp", "trade_id"]].head(10))
    else:
        print(" No duplicate message_ids")

    # Check for duplicate trade_ids
    duplicate_trades = df_validation[df_validation.duplicated(subset=["trade_id"], keep=False)]

    print(f"\nDuplicate trade_ids: {len(duplicate_trades)}")

    if len(duplicate_trades) > 0:
        print(" Found duplicate trade IDs:")
        display(duplicate_trades[["trade_id", "symbol", "timestamp", "message_id"]].head(10))
    else:
        print(" No duplicate trade_ids")

    # Store for quality score
    has_duplicates = len(duplicate_msgs) > 0 or len(duplicate_trades) > 0

else:
    print(" No data available for duplicate detection")
    has_duplicates = False

In [ ]:
# Data quality summary
print("\n" + "=" * 60)
print("DATA QUALITY REPORT")
print("=" * 60 + "\n")

if df_validation is not None and len(df_validation) > 0:
    print("Dataset:")
    print(f"  Total trades analyzed: {len(df_validation):,}")
    print(f"  Unique symbols: {df_validation['symbol'].nunique()}")
    print(f"  Time range: {df_validation['timestamp'].min()} to {df_validation['timestamp'].max()}")

    print("\nQuality Checks:")

    # Schema validation
    schema_pass = all_valid if 'all_valid' in locals() else True
    print(f"  {'' if schema_pass else ''} Schema validation: {'All fields present and valid' if schema_pass else 'Some fields have issues'}")

    # Sequence gaps
    gaps_pass = len(gaps_detected) == 0 if 'gaps_detected' in locals() else True
    print(f"  {'' if gaps_pass else ''} Sequence gaps: {len(gaps_detected) if 'gaps_detected' in locals() else 0} symbols with gaps")

    # Duplicates
    dup_pass = not has_duplicates
    print(f"  {'' if dup_pass else ''} Duplicate detection: {'No duplicates' if dup_pass else 'Duplicates found'}")

    # Calculate quality score
    score = 100
    if not schema_pass:
        score -= 30
    if not gaps_pass:
        score -= 10  # Gaps are normal in streaming
    if not dup_pass:
        score -= 40

    # Display score with color
    if score >= 90:
        score_color = "green"
        rating = "EXCELLENT"
    elif score >= 70:
        score_color = "yellow"
        rating = "GOOD"
    elif score >= 50:
        score_color = "yellow"
        rating = "FAIR"
    else:
        score_color = "red"
        rating = "POOR"

    print(f"\n[bold {score_color}]Data Quality Score: {score}/100 ({rating})[/bold {score_color}]")

    # Recommendations
    if score < 100:
        print("\nRecommendations:")
        if not schema_pass:
            print("  • Fix schema validation issues (missing or null fields)")
        if not dup_pass:
            print("  • Investigate duplicate detection logic in consumer")
        if not gaps_pass:
            print("  • Sequence gaps are normal but monitor gap frequency")
    else:
        print("\n Data quality is excellent! Pipeline is operating correctly.")
else:
    print(" No data available for quality report")

print("\n" + "=" * 60)

## 13. Vendor Data Analysis

The v2 schema's `vendor_data` field is key to multi-source compatibility. Let's analyze how Binance-specific fields are preserved:
1. **Field coverage** - Which vendor fields are present
2. **Field statistics** - Coverage percentage across trades
3. **Example data** - Sample vendor_data entries

In [ ]:
# Parse vendor_data and analyze field coverage
print("\nAnalyzing vendor_data fields...\n")

import json

if df_validation is not None and len(df_validation) > 0:
    # Parse vendor_data JSON for all trades
    df_validation["vendor_parsed"] = df_validation["vendor_data"].apply(
        lambda x: json.loads(x) if pd.notna(x) and x else {}
    )

    # Collect all unique vendor fields
    all_vendor_fields = set()
    for vendor_dict in df_validation["vendor_parsed"]:
        all_vendor_fields.update(vendor_dict.keys())

    print(f" Found {len(all_vendor_fields)} unique Binance-specific fields\n")
    print("Binance vendor_data fields:")
    for field in sorted(all_vendor_fields):
        print(f"  • {field}")

    # Calculate field coverage
    field_counts = {}
    for field in all_vendor_fields:
        count = sum(1 for vd in df_validation["vendor_parsed"] if field in vd)
        field_counts[field] = count

    # Create coverage table
    coverage_table = Table(title="Vendor Field Coverage Statistics", show_header=True)
    coverage_table.add_column("Field", style="cyan")
    coverage_table.add_column("Count", style="green")
    coverage_table.add_column("Coverage %", style="yellow")

    total_trades = len(df_validation)
    for field in sorted(field_counts.keys(), key=lambda x: field_counts[x], reverse=True):
        count = field_counts[field]
        percentage = (count / total_trades) * 100
        coverage_table.add_row(
            field,
            f"{count:,}",
            f"{percentage:.1f}%"
        )

    print("\n")
    print(coverage_table)

else:
    print(" No data available for vendor_data analysis")

In [ ]:
# Show example vendor_data entries
print("\nExample vendor_data entries:\n")

if df_validation is not None and len(df_validation) > 0:
    # Show 3 example vendor_data entries
    num_examples = min(3, len(df_validation))

    for i in range(num_examples):
        trade = df_validation.iloc[i]
        vendor_data = trade["vendor_parsed"]

        print(f"Trade {i+1} ({trade['symbol']}):")
        print("  Core fields:")
        print(f"    • Price: {trade['price']} {trade['currency']}")
        print(f"    • Quantity: {trade['quantity']}")
        print(f"    • Side: {trade['side']}")
        print(f"    • Timestamp: {trade['timestamp']}")

        print("  Vendor data (Binance-specific):")
        for key, value in vendor_data.items():
            print(f"    • {key}: {value}")
        print()

    print("Key Insight:")
    print("  • Core fields (price, quantity, side) are normalized across all exchanges")
    print("  • vendor_data preserves exchange-specific fields without modification")
    print("  • Same v2 schema works for ASX (equities), Binance (crypto), and future exchanges")
    print("  • Enables advanced analysis using exchange-specific metadata")

else:
    print(" No data available for vendor_data examples")

## 14. Architecture Summary & Links

### What We Demonstrated

This notebook showcased the complete end-to-end Binance cryptocurrency streaming pipeline:

| Component | Description | Status |
|-----------|-------------|--------|
| **Binance WebSocket** | Live crypto trades streaming (BTC, ETH) | ✓ Validated |
| **Kafka Broker** | Message queueing with 4 partitions | ✓ Validated |
| **Schema Registry** | V2 Avro schema management | ✓ Validated |
| **Consumer Pipeline** | Batch processing (Kafka → Iceberg) | ✓ Validated |
| **Iceberg Lakehouse** | ACID transactions, Parquet storage | ✓ Validated |
| **Query Engine** | DuckDB analytical queries | ✓ Validated |
| **V2 Hybrid Schema** | Standard fields + vendor_data map | ✓ Validated |
| **Multi-Asset-Class** | Crypto + Equities support | ✓ Validated |
| **Prometheus Metrics** | Real-time pipeline monitoring | ✓ Validated |
| **Data Quality** | Schema, sequence, duplicate checks | ✓ Validated |

---

### Key Achievements (Phase 2 Prep)

**V2 Schema Evolution:**
- Industry-standard core fields (symbol, price, quantity, side, etc.)
- `vendor_data` JSON map for exchange-specific fields
- Works across ASX (equities) and Binance (crypto)
- Future-proof for Coinbase, Kraken, and other exchanges

**Multi-Source Capability:**
- Batch ingestion: ASX CSV files
- Streaming ingestion: Binance WebSocket
- Same v2 schema for both sources
- Unified query interface

**Production-Grade Features:**
- SSL/TLS support
- Exponential backoff and circuit breakers
- Prometheus metrics (7 Binance-specific metrics)
- Sub-second query performance
- 138+ msg/s consumer throughput

**Data Guarantees:**
- ACID transactions via Apache Iceberg
- Exactly-once semantics with idempotent producers
- Schema evolution support
- Time-travel queries

---

### Performance Metrics

From E2E validation session (2026-01-13):

- **Messages Received**: 69,666+ trades from Binance
- **Messages Written**: 5,000+ trades to Iceberg
- **Consumer Throughput**: 138 msg/s
- **Query Latency**: Sub-second for 5,000 records
- **Uptime**: 0 connection errors during demo

---

### Links & Resources

**Local Services:**
- **Prometheus Metrics**: [http://localhost:9090](http://localhost:9090)
- **Grafana Dashboards**: [http://localhost:3000](http://localhost:3000) (admin/admin)
- **Kafka UI**: [http://localhost:8080](http://localhost:8080)
- **MinIO Console**: [http://localhost:9001](http://localhost:9001) (minioadmin/minioadmin)
- **Schema Registry**: [http://localhost:8081](http://localhost:8081)

**Documentation:**
- Phase 2 Prep README: `docs/phases/phase-2-prep/README.md`
- E2E Demo Success Summary: `docs/operations/e2e-demo-success-summary.md`
- V2 Schema Design: `docs/architecture/schema-design-v2.md`
- Streaming Architecture: `docs/architecture/streaming-architecture.md`

**Scripts:**
- Binance Streaming: `scripts/binance_stream.py`
- Infrastructure Init: `scripts/init_e2e_demo.py`
- Consumer: `src/k2/ingestion/consumer.py`
- Query Engine: `src/k2/query/engine.py`

---

### Next Steps

**Explore the Platform:**
1. View metrics in Prometheus: [http://localhost:9090](http://localhost:9090)
2. Create Grafana dashboards for real-time monitoring
3. Query trades via the REST API (coming in Phase 2 Demo Enhancements)
4. Try the ASX historical data demo: `notebooks/demo.ipynb`

**Extend the Pipeline:**
1. Add more crypto symbols (BNB, SOL, ADA, etc.)
2. Stream from multiple exchanges (see Section 15 TODO)
3. Build custom analytics queries
4. Set up alerting rules in Prometheus

**Production Deployment:**
1. Enable SSL certificate verification
2. Add Kafka broker replication (3+ brokers)
3. Deploy distributed Iceberg catalog
4. Set up authentication and authorization
5. Configure data retention policies

---

### Architecture Highlights

**Why This Architecture?**

1. **Apache Kafka**: Industry-standard streaming platform, horizontal scalability, fault tolerance
2. **Apache Iceberg**: ACID guarantees, time-travel queries, schema evolution, S3 compatibility
3. **DuckDB**: Embedded analytics, Parquet-native, sub-second queries, no separate server
4. **Avro + Schema Registry**: Schema evolution, compact serialization, version management
5. **Prometheus + Grafana**: Real-time metrics, alerting, visualization

**Design Principles:**

- **Separation of Concerns**: Ingestion → Storage → Query
- **Idempotency**: Safe retries, exactly-once semantics
- **Schema Evolution**: Forward and backward compatibility
- **Multi-Tenancy**: Asset classes, exchanges, symbols
- **Observability**: Metrics, logs, traces

---

**Congratulations!** You've completed the Binance E2E streaming pipeline demo.

This platform demonstrates **Principal/Staff-level data engineering** with production-grade architecture, multi-source compatibility, and sub-second query performance.

## 15. TODO - Cross-Exchange Comparison (Future Enhancement)

### Goal

Compare cryptocurrency prices across multiple exchanges (Binance, Coinbase, Kraken) to:
- Demonstrate the platform's multi-source capability
- Identify arbitrage opportunities
- Validate v2 schema works across exchanges
- Show value of vendor_data for exchange-specific analysis

---

### Implementation Plan

#### 1. Add Coinbase WebSocket Client

**File**: `src/k2/ingestion/coinbase_client.py`

Similar to `BinanceWebSocketClient`, implement:
- WebSocket connection to `wss://ws-feed.exchange.coinbase.com`
- Subscribe to `matches` channel for BTC-USD, ETH-USD
- Convert Coinbase messages to v2 schema
- Store Coinbase-specific fields in `vendor_data`:
  - `maker_order_id`, `taker_order_id`
  - `sequence`, `product_id`
  - `maker_fee`, `taker_fee`

**Example Coinbase Message:**
```json
{
  "type": "match",
  "trade_id": 12345678,
  "maker_order_id": "abc123",
  "taker_order_id": "def456",
  "side": "buy",
  "size": "0.05",
  "price": "65000.00",
  "product_id": "BTC-USD",
  "sequence": 987654321,
  "time": "2024-01-01T00:00:00.000000Z"
}
```

**V2 Conversion:**
- `symbol`: "BTCUSD" (normalized)
- `exchange`: "COINBASE"
- `asset_class`: "crypto"
- `vendor_data`: Coinbase-specific fields

---

#### 2. Add Kraken WebSocket Client

**File**: `src/k2/ingestion/kraken_client.py`

Similar implementation for Kraken:
- WebSocket connection to `wss://ws.kraken.com`
- Subscribe to `trade` channel for XBT/USD, ETH/USD
- Convert Kraken messages to v2 schema
- Store Kraken-specific fields in `vendor_data`:
  - `order_type` (market, limit)
  - `misc` (additional flags)

---

#### 3. Create Multi-Exchange Streaming Service

**File**: `scripts/multi_exchange_stream.py`

Connect to multiple exchanges simultaneously:

```python
import asyncio
from k2.ingestion.binance_client import BinanceWebSocketClient
from k2.ingestion.coinbase_client import CoinbaseWebSocketClient
from k2.ingestion.kraken_client import KrakenWebSocketClient
from k2.ingestion.producer import MarketDataProducer

async def stream_all_exchanges():
    producer = MarketDataProducer(schema_version="v2")
    
    # Create clients
    binance = BinanceWebSocketClient(symbols=["BTCUSDT", "ETHUSDT"], producer=producer)
    coinbase = CoinbaseWebSocketClient(symbols=["BTC-USD", "ETH-USD"], producer=producer)
    kraken = KrakenWebSocketClient(symbols=["XBT/USD", "ETH/USD"], producer=producer)
    
    # Connect all
    await asyncio.gather(
        binance.connect(),
        coinbase.connect(),
        kraken.connect(),
    )
    
    # Stream indefinitely
    await asyncio.Event().wait()

if __name__ == "__main__":
    asyncio.run(stream_all_exchanges())
```

**Kafka Topics:**
- Option 1: Single topic `market.crypto.trades` (all exchanges)
- Option 2: Per-exchange topics:
  - `market.crypto.trades.binance`
  - `market.crypto.trades.coinbase`
  - `market.crypto.trades.kraken`

---

#### 4. Notebook Enhancements

Add new cells to this notebook:

**Cell: Query trades from all exchanges**
```python
# Query BTCUSDT from all exchanges
trades_binance = engine.query_trades(symbol="BTCUSDT", exchange="BINANCE", limit=1000)
trades_coinbase = engine.query_trades(symbol="BTCUSD", exchange="COINBASE", limit=1000)
trades_kraken = engine.query_trades(symbol="XBTUSD", exchange="KRAKEN", limit=1000)

# Normalize symbol names for comparison
df_binance = pd.DataFrame(trades_binance)
df_coinbase = pd.DataFrame(trades_coinbase)
df_kraken = pd.DataFrame(trades_kraken)
```

**Cell: Calculate price spreads**
```python
# Calculate average prices
avg_binance = df_binance["price"].mean()
avg_coinbase = df_coinbase["price"].mean()
avg_kraken = df_kraken["price"].mean()

# Calculate spreads (arbitrage opportunities)
spread_binance_coinbase = abs(avg_binance - avg_coinbase)
spread_binance_kraken = abs(avg_binance - avg_kraken)
spread_coinbase_kraken = abs(avg_coinbase - avg_kraken)

print(f"Price Spreads:")
print(f"  Binance-Coinbase: ${spread_binance_coinbase:.2f}")
print(f"  Binance-Kraken: ${spread_binance_kraken:.2f}")
print(f"  Coinbase-Kraken: ${spread_coinbase_kraken:.2f}")
```

**Cell: Visualize price comparison**
```python
fig, ax = plt.subplots(figsize=(14, 6))

# Plot prices from all exchanges
ax.plot(df_binance["timestamp"], df_binance["price"], label="Binance", alpha=0.7)
ax.plot(df_coinbase["timestamp"], df_coinbase["price"], label="Coinbase", alpha=0.7)
ax.plot(df_kraken["timestamp"], df_kraken["price"], label="Kraken", alpha=0.7)

ax.set_xlabel("Time (UTC)")
ax.set_ylabel("Price (USD)")
ax.set_title("BTC Price Comparison Across Exchanges")
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()
```

**Cell: Heatmap of spreads over time**
```python
# Aggregate prices by 1-minute buckets
df_binance["bucket"] = df_binance["timestamp"].dt.floor("1min")
df_coinbase["bucket"] = df_coinbase["timestamp"].dt.floor("1min")
df_kraken["bucket"] = df_kraken["timestamp"].dt.floor("1min")

# Calculate mean price per bucket
price_binance = df_binance.groupby("bucket")["price"].mean()
price_coinbase = df_coinbase.groupby("bucket")["price"].mean()
price_kraken = df_kraken.groupby("bucket")["price"].mean()

# Calculate spread matrix
spread_matrix = pd.DataFrame({
    "Binance-Coinbase": abs(price_binance - price_coinbase),
    "Binance-Kraken": abs(price_binance - price_kraken),
    "Coinbase-Kraken": abs(price_coinbase - price_kraken),
})

# Plot heatmap
import seaborn as sns
sns.heatmap(spread_matrix.T, cmap="YlOrRd", cbar_kws={"label": "Spread (USD)"})
plt.title("Price Spreads Over Time (1-minute buckets)")
plt.xlabel("Time")
plt.ylabel("Exchange Pair")
plt.show()
```

---

### Benefits

**Demonstrates Platform Capabilities:**
- Multi-source data ingestion (3+ exchanges)
- V2 schema flexibility (works across all exchanges)
- vendor_data preserves exchange-specific fields
- Unified query interface for cross-exchange analysis

**Real-World Use Cases:**
- **Arbitrage Detection**: Identify profitable price differences
- **Market Microstructure**: Study price formation across venues
- **Liquidity Analysis**: Compare trade volumes and spreads
- **Best Execution**: Route orders to best-priced exchange

**Technical Validation:**
- Schema evolution: Same v2 schema for Binance, Coinbase, Kraken
- Query performance: Sub-second queries across millions of trades
- Data quality: Validate consistency across exchanges
- Vendor data: Preserve exchange-specific fields for advanced analysis

---

### Example Output

**Price Comparison Table:**

| Exchange | Avg Price | Min Price | Max Price | Spread % | Volume |
|----------|-----------|-----------|-----------|----------|--------|
| Binance  | 65,123.45 | 65,000.00 | 65,250.00 | 0.38% | 150.25 BTC |
| Coinbase | 65,145.20 | 65,020.00 | 65,270.00 | 0.38% | 98.50 BTC |
| Kraken   | 65,110.80 | 64,990.00 | 65,230.00 | 0.37% | 75.30 BTC |

**Arbitrage Opportunities:**

- Binance → Coinbase: $21.75 spread (0.03%)
- Kraken → Coinbase: $34.40 spread (0.05%) ← **Best opportunity**
- Binance → Kraken: $12.65 spread (0.02%)

**Transaction costs**: ~0.1% (typical exchange fees)
**Profitable if**: Spread > 0.1%
**Result**: Kraken → Coinbase arbitrage is profitable (0.05% > 0.1%)

---

### Next Steps to Implement

1. **Research APIs**: Study Coinbase and Kraken WebSocket APIs
2. **Implement clients**: Build CoinbaseWebSocketClient and KrakenWebSocketClient
3. **Test conversion**: Validate v2 schema conversion for each exchange
4. **Deploy streaming**: Run multi_exchange_stream.py in production
5. **Create notebook**: Add cross-exchange comparison cells
6. **Document findings**: Write report on arbitrage opportunities

**Estimated Time**: 8-12 hours
**Priority**: Medium (nice-to-have, demonstrates platform flexibility)
**Dependencies**: Phase 2 Prep complete (v2 schema + Binance streaming)

---

**Note**: This is documented as future work. The platform's v2 schema and architecture are already designed to support multiple exchanges - implementation is straightforward once exchange clients are built.